In [53]:
## Show the total number of observations per species
## Which species has less observations?
## Which species has more observations?
## Fill the conservation status of the species
## Show the number of species in each conservation_status

import pandas as pd

obs = pd.read_csv('observations.csv')
species = pd.read_csv('species_info.csv')

print(obs.head())
print(species.head())

print(obs.count())
print(species.count())

            scientific_name                            park_name  observations
0        Vicia benghalensis  Great Smoky Mountains National Park            68
1            Neovison vison  Great Smoky Mountains National Park            77
2         Prunus subcordata               Yosemite National Park           138
3      Abutilon theophrasti                  Bryce National Park            84
4  Githopsis specularioides  Great Smoky Mountains National Park            85
  category                scientific_name  \
0   Mammal  Clethrionomys gapperi gapperi   
1   Mammal                      Bos bison   
2   Mammal                     Bos taurus   
3   Mammal                     Ovis aries   
4   Mammal                 Cervus elaphus   

                                        common_names conservation_status  
0                           Gapper's Red-Backed Vole                 NaN  
1                              American Bison, Bison                 NaN  
2  Aurochs, Aurochs, Domestic 

In [83]:
## Show the total number of observations per species

obs_grouped = obs.groupby('scientific_name', as_index=False)['observations'].sum()
obs_grouped_sorted = obs_grouped.sort_values('observations', ascending=False)

print(obs_grouped_sorted.head())


            scientific_name  observations
4993  Streptopelia decaocto          1785
2452         Holcus lanatus          1780
1103      Castor canadensis          1725
2555   Hypochaeris radicata          1720
4179          Puma concolor          1711


In [85]:
## Which species has less observations?

less_viewed_species = obs_grouped_sorted['scientific_name'].iloc[-1]
less_views = obs_grouped_sorted['observations'].iloc[-1]
print('The species with less observations is ' + less_viewed_species + ' with ' + str(less_views) + ' views.')

The species with less observations is Grus americana with 131 views.


In [89]:
## Which species has more observations?

most_viewed_species = obs_grouped_sorted['scientific_name'].iloc[0]
most_views = obs_grouped_sorted['observations'].iloc[0]
print('The species with less observations is ' + most_viewed_species + ' with ' + str(most_views) + ' views.')

The species with less observations is Streptopelia decaocto with 1785 views.


In [129]:
## Fill the conservation status of the species
merged_df = pd.merge(obs_grouped_sorted, species, on='scientific_name', how='inner')

##print(merged_df.head())

##print(obs_grouped_sorted[obs_grouped_sorted.duplicated(subset=['scientific_name'])])
print(species.duplicated(subset=['scientific_name']).sum())
print(species.duplicated(subset=['scientific_name','conservation_status']).sum())

print(species[species['scientific_name']=='Cervus elaphus'])

283
281
     category scientific_name        common_names conservation_status
4      Mammal  Cervus elaphus       Wapiti Or Elk                 NaN
3017   Mammal  Cervus elaphus  Rocky Mountain Elk                 NaN
